# Running actual hypothesis test with pre-computed standard errors

In [80]:
import scanpy as sc
import scipy.stats as stats
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import string
import random
# import estimators as memento

import seaborn as sns
import tiledb
import statsmodels.api as sm
from pymare import estimators, core


### Read estimators

In [119]:
cube_path = '../../estimators_cube'

In [120]:
# seems to be a bug in the cube generation code that names columns incorrectly
estimators = tiledb.open(cube_path).df[:].rename(columns={'feature_id':'cell_type', 'cell_type':'dataset_id', 'dataset_id':'feature_id'})

In [121]:
estimators.head(2)

,cell_type,dataset_id,feature_id,assay,suspension_type,donor_id,disease,sex,nnz,n_obs,min,max,sum,mean,sem,var,sev,selv
0,CD14-positive monocyte,1a2e3350-28a8-4f49-b33c-5b67ceb001f6,ENSG00000000419,10x 5' v1,cell,F51,normal,female,328.0,8763.0,1.0,3.0,360.0,0.028311,0.001594,0.0,0.0,0.0
1,CD14-positive monocyte,1a2e3350-28a8-4f49-b33c-5b67ceb001f6,ENSG00000000457,10x 5' v1,cell,F51,normal,female,89.0,8763.0,1.0,2.0,91.0,0.007215,0.000761,0.0,0.0,0.0


In [122]:
# Fake cell types
estimators['cell_type'] = np.random.choice(['A', 'B'], size=estimators.shape[0])

### Setup DE

In [123]:
CUBE_TILEDB_DIMS_OBS = [
    "cell_type",
    "dataset_id",
]

CUBE_TILEDB_ATTRS_OBS = [
    "assay",
    "suspension_type",
    "donor_id",
    "disease",
    "sex"
]

CUBE_LOGICAL_DIMS_OBS = CUBE_TILEDB_DIMS_OBS + CUBE_TILEDB_ATTRS_OBS

DE_TREATMENT = 'cell_type'
DE_COVARIATES = ['dataset_id', 'donor_id', 'assay']
DE_VARIABLES = [DE_TREATMENT] + DE_COVARIATES

In [124]:
names = estimators[DE_TREATMENT].copy()
for col in DE_COVARIATES:
    names += '_' + estimators[col]
estimators['group_name'] = names.tolist()

In [150]:
features = estimators['feature_id'].drop_duplicates().tolist()
groups = estimators.drop_duplicates(subset='group_name').set_index('group_name')

In [218]:
design = pd.get_dummies(groups[DE_VARIABLES], drop_first=True).astype(int)

In [219]:
mean = estimators.pivot(index='group_name', columns='feature_id', values='mean').fillna(1e-3)
se_mean = estimators.pivot(index='group_name', columns='feature_id', values='sem').fillna(1e-4)
cell_counts = groups['n_obs'].sort_index().values

### Peform DE for each gene

In [204]:
def de_wls(X, y, n, v):
    
    from sklearn.linear_model import LinearRegression
    
    # fit WLS using sample_weights
    WLS = LinearRegression()
    WLS.fit(X, y, sample_weight=n)

    coef = WLS.coef_[0]

    W = np.diag(1/v)
    beta_var_hat = np.diag(np.linalg.pinv(X.T@W@X))
    se = np.sqrt( beta_var_hat[0] )

    z = coef/se
    pv = stats.norm.sf(np.abs(z))*2
    
    return coef, z, pv

In [214]:
%%time
de_result = []
for feature in features[:1000]: # Can be vectorized heavily, showing for 1K genes
    
    m = mean[feature].values
    sem = se_mean[feature].values
    
    # Transform to log space (alternatively can resample in log space)
    lm = np.log(m)
    selm = (np.log(m+sem)-np.log(m-sem))/2
    
    coef, z, pv = de_wls(design.values, lm, cell_counts, selm**2)
    de_result.append((feature, coef, z, pv))
    
de_result = pd.DataFrame(de_result, columns=['feature_id','coef', 'z', 'pval']).set_index('feature_id')

CPU times: user 1min 34s, sys: 3min 25s, total: 4min 59s
Wall time: 25.2 s


### Save DE result

In [ ]:
de_result.to_csv('de_result.csv')